# CPP

## Single-File

In [1]:
%%bash
cat > main.cpp << 'EOF'
#include <armadillo>
#include <iostream>
#include <chrono>
#include <iomanip>
#include <cmath>

using namespace arma;
using namespace std;
using namespace std::chrono;

// normalize_zero returns 0.0 if x is very close to zero
double normalize_zero(double x, double tol = 1e-12) {
    return (fabs(x) < tol) ? 0.0 : x;
}

int main() {
    // Create a simple dataset:
    // X: a 5x1 matrix representing [1, 2, 3, 4, 5]
    // y: a 5x1 vector representing [2, 4, 6, 8, 10] (i.e. y = 2 * x)
    mat X(5, 1);
    X(0, 0) = 1.0;
    X(1, 0) = 2.0;
    X(2, 0) = 3.0;
    X(3, 0) = 4.0;
    X(4, 0) = 5.0;

    vec y(5);
    y(0) = 2.0;
    y(1) = 4.0;
    y(2) = 6.0;
    y(3) = 8.0;
    y(4) = 10.0;

    // Augment X with a column of ones for the intercept
    mat X_aug = join_horiz(ones<vec>(X.n_rows), X);

    // Time the training (solving for theta)
    auto train_start = high_resolution_clock::now();
    // Solve the normal equation: theta = (X_aug^T * X_aug)^-1 * (X_aug^T * y)
    vec theta = solve(X_aug, y);
    auto train_end = high_resolution_clock::now();
    duration<double, milli> train_duration = train_end - train_start;

    // Normalize theta values so that very-small numbers print as 0.0000
    for (uword i = 0; i < theta.n_elem; i++) {
        theta(i) = normalize_zero(theta(i));
    }

    // Print learned parameters with 4 decimal places per element
    cout << "Learned parameters (theta): [";
    for (uword i = 0; i < theta.n_elem; i++) {
        if (i > 0)
            cout << ", ";
        cout << fixed << setprecision(4) << theta(i);
    }
    cout << "]" << endl;

    // Print training time with 6 decimal places
    cout << "Training time: " << fixed << setprecision(6) << train_duration.count() << " ms" << endl;

    // Predict for a new data point: x = 6
    mat X_new(1, 1);
    X_new(0, 0) = 6.0;
    mat X_new_aug = join_horiz(ones<vec>(X_new.n_rows), X_new);

    auto pred_start = high_resolution_clock::now();
    vec y_pred = X_new_aug * theta;
    auto pred_end = high_resolution_clock::now();
    duration<double, milli> pred_duration = pred_end - pred_start;

    cout << "Prediction for x = 6: " << fixed << setprecision(4) << y_pred[0] << endl;
    cout << "Prediction time: " << fixed << setprecision(6) << pred_duration.count() << " ms" << endl;

    return 0;
}
EOF

In [2]:
%%bash
g++ -O2 -std=c++11 main.cpp -o linear_regression -larmadillo -llapack -lblas

In [3]:
# %%bash
# g++ -O2 -std=c++11 main.cpp -o linear_regression -larmadillo -lopenblas

In [4]:
!./linear_regression

Learned parameters (theta): [0.0000, 2.0000]
Training time: 16.741325 ms
Prediction for x = 6: 12.0000
Prediction time: 0.009121 ms


## Multi-File

In [5]:
%%bash
cat > dataset.h << 'EOF'
#ifndef DATASET_H
#define DATASET_H

#include <armadillo>

// CreateDataset fills X and y with the dataset values
void CreateDataset(arma::mat &X, arma::vec &y);

#endif
EOF

In [6]:
%%bash
cat > dataset.cpp << 'EOF'
#include "dataset.h"

void CreateDataset(arma::mat &X, arma::vec &y) {
    // Create X as a 5x1 matrix representing [1, 2, 3, 4, 5]
    X.set_size(5, 1);
    X(0, 0) = 1.0;
    X(1, 0) = 2.0;
    X(2, 0) = 3.0;
    X(3, 0) = 4.0;
    X(4, 0) = 5.0;
    // Create y as a 5-element vector representing [2, 4, 6, 8, 10]
    y.set_size(5);
    y(0) = 2.0;
    y(1) = 4.0;
    y(2) = 6.0;
    y(3) = 8.0;
    y(4) = 10.0;
}
EOF

In [7]:
%%bash
cat > model.h << 'EOF'
#ifndef MODEL_H
#define MODEL_H

#include <armadillo>

// BuildAugmentedMatrix returns an augmented matrix from X
// The augmented matrix has a column of ones prepended to X
arma::mat BuildAugmentedMatrix(const arma::mat &X);

#endif
EOF

In [8]:
%%bash
cat > model.cpp << 'EOF'
#include "model.h"

arma::mat BuildAugmentedMatrix(const arma::mat &X) {
    // Get the number of rows from X
    arma::uword n = X.n_rows;
    // Return a new matrix which is the horizontal concatenation of a column of ones and X
    return arma::join_horiz(arma::ones<arma::vec>(n), X);
}
EOF

In [9]:
%%bash
cat > train.h << 'EOF'
#ifndef TRAIN_H
#define TRAIN_H

#include <armadillo>
#include <chrono>

// TrainModel trains the model using the augmented matrix X_aug and target vector y
// It returns the computed theta and sets trainDuration (in milliseconds)
arma::vec TrainModel(const arma::mat &X_aug, const arma::vec &y, double &trainDuration);

#endif
EOF

In [10]:
%%bash
cat > train.cpp << 'EOF'
#include "train.h"

arma::vec TrainModel(const arma::mat &X_aug, const arma::vec &y, double &trainDuration) {
    auto start = std::chrono::high_resolution_clock::now();
    // Solve the normal equation: theta = (X_aug^T * X_aug)^-1 * (X_aug^T * y)
    arma::vec theta = arma::solve(X_aug, y);
    auto end = std::chrono::high_resolution_clock::now();
    std::chrono::duration<double, std::milli> dur = end - start;
    trainDuration = dur.count();
    return theta;
}
EOF

In [11]:
%%bash
cat > predict.h << 'EOF'
#ifndef PREDICT_H
#define PREDICT_H

#include <armadillo>
#include <chrono>

// PredictValue predicts the output for a new input x_new using theta
// It returns the predicted value and sets predDuration (in milliseconds)
double PredictValue(const arma::vec &theta, double x_new, double &predDuration);

#endif
EOF

In [12]:
%%bash
cat > predict.cpp << 'EOF'
#include "predict.h"

double PredictValue(const arma::vec &theta, double x_new, double &predDuration) {
    // Create an augmented row for x_new: first element is 1, second is x_new
    arma::mat X_new(1, 2);
    X_new(0, 0) = 1.0;
    X_new(0, 1) = x_new;
    auto start = std::chrono::high_resolution_clock::now();
    arma::vec y_pred = X_new * theta;
    auto end = std::chrono::high_resolution_clock::now();
    std::chrono::duration<double, std::milli> dur = end - start;
    predDuration = dur.count();
    return y_pred[0];
}
EOF

In [13]:
%%bash
cat > main.cpp << 'EOF'
#include <armadillo>
#include <iostream>
#include <iomanip>
#include "dataset.h"
#include "model.h"
#include "train.h"
#include "predict.h"

// normalizeZero returns 0.0 if x is very close to zero
double normalizeZero(double x, double tol = 1e-12) {
    return (std::abs(x) < tol) ? 0.0 : x;
}

int main() {
    arma::mat X;
    arma::vec y;
    // Create the dataset
    CreateDataset(X, y);

    // Build the augmented matrix from X
    arma::mat X_aug = BuildAugmentedMatrix(X);

    double trainTime = 0.0;
    arma::vec theta = TrainModel(X_aug, y, trainTime);

    // Normalize theta values to avoid printing -0.0000
    for (arma::uword i = 0; i < theta.n_elem; i++) {
        theta(i) = normalizeZero(theta(i));
    }

    // Print learned parameters (theta) with fixed 4 decimal places
    std::cout << "Learned parameters (theta): [";
    for (arma::uword i = 0; i < theta.n_elem; i++) {
        if (i > 0)
            std::cout << ", ";
        std::cout << std::fixed << std::setprecision(4) << theta(i);
    }
    std::cout << "]" << std::endl;
    std::cout << "Training time: " << std::fixed << std::setprecision(6) << trainTime << " ms" << std::endl;

    double predTime = 0.0;
    double prediction = PredictValue(theta, 6.0, predTime);
    std::cout << "Prediction for x = 6: " << std::fixed << std::setprecision(4) << prediction << std::endl;
    std::cout << "Prediction time: " << std::fixed << std::setprecision(6) << predTime << " ms" << std::endl;

    return 0;
}
EOF

In [14]:
%%bash
g++ -O2 -std=c++11 dataset.cpp model.cpp train.cpp predict.cpp main.cpp -o linear_regression -larmadillo -llapack -lblas

In [15]:
!./linear_regression

Learned parameters (theta): [0.0000, 2.0000]
Training time: 0.200274 ms
Prediction for x = 6: 12.0000
Prediction time: 0.002297 ms


# Rust

## Single-File

In [16]:
%%bash
curl https://sh.rustup.rs -sSf | sh -s -- -y
export PATH="$HOME/.cargo/bin:$PATH"
rustc --version


  stable-x86_64-unknown-linux-gnu installed - rustc 1.84.1 (e71f9a9a9 2025-01-27)


Rust is installed now. Great!

To get started you may need to restart your current shell.
This would reload your PATH environment variable to include
Cargo's bin directory ($HOME/.cargo/bin).

To configure your current shell, you need to source
the corresponding env file under $HOME/.cargo.

This is usually done by running one of the following (note the leading DOT):
. "$HOME/.cargo/env"            # For sh/bash/zsh/ash/dash/pdksh
source "$HOME/.cargo/env.fish"  # For fish
rustc 1.84.1 (e71f9a9a9 2025-01-27)


info: downloading installer
info: profile set to 'default'
info: default host triple is x86_64-unknown-linux-gnu
info: syncing channel updates for 'stable-x86_64-unknown-linux-gnu'
info: latest update on 2025-01-30, rust version 1.84.1 (e71f9a9a9 2025-01-27)
info: downloading component 'cargo'
info: downloading component 'clippy'
info: downloading component 'rust-docs'
info: downloading component 'rust-std'
info: downloading component 'rustc'
info: downloading component 'rustfmt'
info: installing component 'cargo'
info: installing component 'clippy'
info: installing component 'rust-docs'
info: installing component 'rust-std'
info: installing component 'rustc'
info: installing component 'rustfmt'
info: default toolchain set to 'stable-x86_64-unknown-linux-gnu'


In [17]:
%%bash
rm -rf rust_linear_regression
$HOME/.cargo/bin/cargo init --bin rust_linear_regression

    Creating binary (application) package
note: see more `Cargo.toml` keys and their definitions at https://doc.rust-lang.org/cargo/reference/manifest.html


In [18]:
%%bash
cd rust_linear_regression
# Append the dependency only if not already present
grep -q 'nalgebra' Cargo.toml || echo 'nalgebra = "0.32.0"' >> Cargo.toml
cd ..

In [19]:
%%bash
mkdir -p rust_linear_regression/src
cat > rust_linear_regression/src/main.rs << 'EOF'
use nalgebra::{DMatrix, DVector};
use std::time::Instant;

fn main() {
    // Create a dataset:
    // x = [1, 2, 3, 4, 5]
    // y = [2, 4, 6, 8, 10] (i.e. y = 2 * x)
    let x_values = vec![1.0, 2.0, 3.0, 4.0, 5.0];
    let y_values = vec![2.0, 4.0, 6.0, 8.0, 10.0];
    let n = x_values.len();

    // Build an augmented matrix (x_aug) directly:
    // The first column is ones (for the intercept), and the second column is x_values
    let x_aug = DMatrix::from_fn(n, 2, |i, j| if j == 0 { 1.0 } else { x_values[i] });

    // Build y as an n x 1 vector
    let y = DVector::from_vec(y_values);

    // Time the training (solving for theta)
    let train_start = Instant::now();
    let xt = x_aug.transpose();
    let xtx = &xt * &x_aug;
    let xty = &xt * &y;
    let theta = xtx.try_inverse().expect("Matrix is invertible") * xty;
    let train_duration = train_start.elapsed();

    // Print theta with 4 decimal places per element
    print!("Learned parameters (theta): [");
    for i in 0..theta.len() {
        if i > 0 {
            print!(", ");
        }
        // Print each element with 4 decimal places
        print!("{:.4}", theta[i]);
    }
    println!("]");
    println!("Training time: {:.4} ms", train_duration.as_secs_f64() * 1000.0);

    // For prediction, create an augmented matrix for x = 6: [1, 6]
    let x_new_aug = DMatrix::from_fn(1, 2, |_, j| if j == 0 { 1.0 } else { 6.0 });
    let pred_start = Instant::now();
    let y_pred = &x_new_aug * &theta;
    let pred_duration = pred_start.elapsed();

    println!("Prediction for x = 6: {:.4}", y_pred[0]);
    println!("Prediction time: {:.4} ms", pred_duration.as_secs_f64() * 1000.0);
}
EOF

In [20]:
%%bash
cd rust_linear_regression
$HOME/.cargo/bin/cargo run --release --quiet
cd ..

Learned parameters (theta): [0.0000, 2.0000]
Training time: 0.0006 ms
Prediction for x = 6: 12.0000
Prediction time: 0.0003 ms


In [21]:
# %%bash
# cd rust_linear_regression
# $HOME/.cargo/bin/cargo build --release --quiet
# ./target/release/rust_linear_regression
# cd ..

## Multi-File

In [22]:
%%bash
curl https://sh.rustup.rs -sSf | sh -s -- -y
export PATH="$HOME/.cargo/bin:$PATH"
rustc --version


  stable-x86_64-unknown-linux-gnu unchanged - rustc 1.84.1 (e71f9a9a9 2025-01-27)


Rust is installed now. Great!

To get started you may need to restart your current shell.
This would reload your PATH environment variable to include
Cargo's bin directory ($HOME/.cargo/bin).

To configure your current shell, you need to source
the corresponding env file under $HOME/.cargo.

This is usually done by running one of the following (note the leading DOT):
. "$HOME/.cargo/env"            # For sh/bash/zsh/ash/dash/pdksh
source "$HOME/.cargo/env.fish"  # For fish
rustc 1.84.1 (e71f9a9a9 2025-01-27)


info: downloading installer
info: profile set to 'default'
info: default host triple is x86_64-unknown-linux-gnu
info: syncing channel updates for 'stable-x86_64-unknown-linux-gnu'
info: default toolchain set to 'stable-x86_64-unknown-linux-gnu'


In [23]:
%%bash
rm -rf rust_linear_regression
$HOME/.cargo/bin/cargo init --bin rust_linear_regression

    Creating binary (application) package
note: see more `Cargo.toml` keys and their definitions at https://doc.rust-lang.org/cargo/reference/manifest.html


In [24]:
%%bash
cd rust_linear_regression
# Append the dependency if not already present
grep -q 'nalgebra' Cargo.toml || echo 'nalgebra = "0.32.0"' >> Cargo.toml
cd ..

In [25]:
%%bash
cat > rust_linear_regression/src/dataset.rs << 'EOF'
pub fn create_dataset() -> (Vec<f64>, Vec<f64>) {
    let x_values = vec![1.0, 2.0, 3.0, 4.0, 5.0];
    let y_values = vec![2.0, 4.0, 6.0, 8.0, 10.0];
    (x_values, y_values)
}
EOF

In [26]:
%%bash
cat > rust_linear_regression/src/model.rs << 'EOF'
use nalgebra::DMatrix;

pub fn build_augmented_matrix(x_values: &Vec<f64>) -> DMatrix<f64> {
    let n = x_values.len();
    // Create an n x 2 matrix:
    // first column: ones (for the intercept)
    // second column: x_values
    DMatrix::from_fn(n, 2, |i, j| if j == 0 { 1.0 } else { x_values[i] })
}
EOF

In [27]:
%%bash
cat > rust_linear_regression/src/train.rs << 'EOF'
use nalgebra::{DMatrix, DVector};
use std::time::Instant;

pub fn train_model(x_aug: &DMatrix<f64>, y_values: &Vec<f64>) -> (DVector<f64>, f64) {
    // Build y as an n x 1 vector (the length is inferred from x_aug.nrows())
    let y = DVector::from_vec(y_values.clone());
    let start = Instant::now();
    let xt = x_aug.transpose();
    let xtx = &xt * x_aug;
    let xty = &xt * &y;
    let theta = xtx.try_inverse().expect("Matrix is invertible") * xty;
    let duration_ms = start.elapsed().as_secs_f64() * 1000.0;
    (theta, duration_ms)
}
EOF

In [28]:
%%bash
cat > rust_linear_regression/src/predict.rs << 'EOF'
use nalgebra::DMatrix;
use std::time::Instant;

pub fn predict(theta: &nalgebra::DVector<f64>, x_new: f64) -> (f64, f64) {
    // Build an augmented row for x_new: [1, x_new]
    let x_new_aug = DMatrix::from_fn(1, 2, |_, j| if j == 0 { 1.0 } else { x_new });
    let start = Instant::now();
    let y_pred = &x_new_aug * theta;
    let duration_ms = start.elapsed().as_secs_f64() * 1000.0;
    (y_pred[0], duration_ms)
}
EOF

In [29]:
%%bash
cat > rust_linear_regression/src/main.rs << 'EOF'
mod dataset;
mod model;
mod train;
mod predict;

use dataset::create_dataset;
use model::build_augmented_matrix;
use train::train_model;
use predict::predict;
use std::fmt::Write;

fn normalize_zero(x: f64, tol: f64) -> f64 {
    if x.abs() < tol { 0.0 } else { x }
}

fn main() {
    // Create the dataset
    let (x_values, y_values) = create_dataset();

    // Build the augmented matrix from x_values
    let x_aug = build_augmented_matrix(&x_values);

    // Train the model and get training time (in ms)
    let (theta, train_time_ms) = train_model(&x_aug, &y_values);

    // Normalize theta values to avoid printing -0.0000
    let tol = 1e-12;
    let theta_norm: Vec<f64> = theta.iter().map(|&x| normalize_zero(x, tol)).collect();

    // Format theta as a string with 4 decimal places per element
    let mut theta_str = String::new();
    theta_str.push('[');
    for (i, v) in theta_norm.iter().enumerate() {
        if i > 0 {
            write!(&mut theta_str, ", ").unwrap();
        }
        write!(&mut theta_str, "{:.4}", v).unwrap();
    }
    theta_str.push(']');

    println!("Learned parameters (theta): {}", theta_str);
    println!("Training time: {:.6} ms", train_time_ms);

    // Predict for a new data point: x = 6
    let (prediction, pred_time_ms) = predict(&theta, 6.0);
    println!("Prediction for x = 6: {:.4}", prediction);
    println!("Prediction time: {:.6} ms", pred_time_ms);
}
EOF

In [30]:
%%bash
cd rust_linear_regression
$HOME/.cargo/bin/cargo run --release --quiet
cd ..

Learned parameters (theta): [0.0000, 2.0000]
Training time: 0.002278 ms
Prediction for x = 6: 12.0000
Prediction time: 0.000168 ms


# Golang

## Single-File

In [31]:
%%bash
sudo apt-get update
sudo apt-get install -y golang
go version

Hit:1 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:2 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:3 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:5 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:6 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:7 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease [18.1 kB]
Get:9 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [1,309 kB]
Hit:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:11 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 Packages [2,907 kB]
Hit:12 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:13 http://archive.ubuntu.com/u

W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 78, <> line 8.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype
dpkg-preconfigure: unable to re-open stdin: 


In [32]:
%%bash
mkdir -p golang_linreg
cd golang_linreg
# Initialize the module if not already created
go mod init example.com/linreg || echo "Module already exists"
# Get Gonum v0.14.0 explicitly
go get gonum.org/v1/gonum/mat@v0.14.0
# Force the entire gonum.org/v1/gonum module to version v0.14.0
go mod edit -replace=gonum.org/v1/gonum=gonum.org/v1/gonum@v0.14.0
# Tidy up the module
go mod tidy
cd ..

go: creating new go.mod: module example.com/linreg
go: downloading gonum.org/v1/gonum v0.14.0
go: added gonum.org/v1/gonum v0.14.0
go: warning: "all" matched no packages


In [33]:
%%bash
cat > golang_linreg/main.go << 'EOF'
package main

import (
	"fmt"
	"math"
	"time"

	"gonum.org/v1/gonum/mat"
)

// normalizeZero returns 0.0 if x is very close to zero
func normalizeZero(x float64, tol float64) float64 {
	if math.Abs(x) < tol {
		return 0.0
	}
	return x
}

func main() {
	// Create a dataset:
	// x = [1, 2, 3, 4, 5]
	// y = [2, 4, 6, 8, 10] (i.e. y = 2 * x)
	xValues := []float64{1, 2, 3, 4, 5}
	yValues := []float64{2, 4, 6, 8, 10}
	n := len(xValues)

	// Augment data: create an n x 2 matrix (first column ones, second column xValues)
	XAug := mat.NewDense(n, 2, nil)
	for i := 0; i < n; i++ {
		XAug.Set(i, 0, 1.0)
		XAug.Set(i, 1, xValues[i])
	}

	// Build y as an n x 1 vector
	y := mat.NewVecDense(n, yValues)

	// Time the training (solving for theta)
	trainStart := time.Now()
	var Xt mat.Dense
	Xt.Mul(XAug.T(), XAug) // Xt = XAug^T * XAug

	var XtInv mat.Dense
	if err := XtInv.Inverse(&Xt); err != nil {
		fmt.Println("Matrix inversion failed:", err)
		return
	}

	var Xty mat.VecDense
	Xty.MulVec(XAug.T(), y)

	var theta mat.VecDense
	theta.MulVec(&XtInv, &Xty)
	trainDuration := time.Since(trainStart)

	// Normalize theta values (so that very small numbers print as 0.0000)
	tol := 1e-12
	data := theta.RawVector().Data
	for i, v := range data {
		data[i] = normalizeZero(v, tol)
	}

	fmt.Printf("Learned parameters (theta): [")
	for i, v := range data {
		if i > 0 {
			fmt.Printf(", ")
		}
		fmt.Printf("%.4f", v)
	}
	fmt.Println("]")

	// Print training time in both milliseconds and microseconds
	trainMillis := float64(trainDuration.Milliseconds())
	trainMicros := float64(trainDuration.Microseconds())
	fmt.Printf("Training time: %.6f ms\n", trainMillis)
	fmt.Printf("Training time: %.6f µs\n", trainMicros)

	// Predict for a new data point: x = 6
	// Create an augmented row for x = 6: [1, 6]
	XNewAug := mat.NewDense(1, 2, []float64{1.0, 6.0})

	predStart := time.Now()
	var yPred mat.Dense
	yPred.Mul(XNewAug, &theta) // yPred is a 1 x 1 matrix
	predDuration := time.Since(predStart)

	predMillis := float64(predDuration.Milliseconds())
	predMicros := float64(predDuration.Microseconds())
	fmt.Printf("Prediction for x = 6: %.4f\n", yPred.At(0, 0))
	fmt.Printf("Prediction time: %.6f ms\n", predMillis)
	fmt.Printf("Prediction time: %.6f µs\n", predMicros)
}
EOF

In [34]:
%%bash
cd golang_linreg
export GO111MODULE=on
go mod tidy && go mod download
go run .
cd ..

Learned parameters (theta): [0.0000, 2.0000]
Training time: 0.000000 ms
Training time: 151.000000 µs
Prediction for x = 6: 12.0000
Prediction time: 0.000000 ms
Prediction time: 1.000000 µs


go: found gonum.org/v1/gonum/mat in gonum.org/v1/gonum v0.0.0-00010101000000-000000000000
go: downloading golang.org/x/exp v0.0.0-20230321023759-10a507213a29


## Multi-File

In [35]:
%%bash
sudo apt-get update
sudo apt-get install -y golang
go version

Hit:1 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:3 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:5 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:7 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists...
Reading package lists...
Building dependency tree...
Reading state information...
golang is already the newest version (2:1.18~0ubuntu2).
0 upgraded, 0 newly installed, 0 to remove and 27 not upgraded.
go version go1.18.1 linux/amd64


W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)


In [36]:
%%bash
rm -rf golang_linreg

In [37]:
%%bash
mkdir -p golang_linreg
cd golang_linreg
# Initialize the module
go mod init example.com/linreg || echo "Module already exists"
# Force using Gonum v0.14.0 (compatible with Go 1.18)
go get gonum.org/v1/gonum/mat@v0.14.0
go mod edit -replace=gonum.org/v1/gonum=gonum.org/v1/gonum@v0.14.0
go mod tidy
cd ..

go: creating new go.mod: module example.com/linreg
go: added gonum.org/v1/gonum v0.14.0
go: warning: "all" matched no packages


In [38]:
%%bash
cat > golang_linreg/dataset.go << 'EOF'
package main

// CreateDataset returns the x-values, y-values, and the number of data points
func CreateDataset() ([]float64, []float64, int) {
	xValues := []float64{1, 2, 3, 4, 5}
	yValues := []float64{2, 4, 6, 8, 10}
	return xValues, yValues, len(xValues)
}
EOF

In [39]:
%%bash
cat > golang_linreg/model.go << 'EOF'
package main

import "gonum.org/v1/gonum/mat"

// BuildAugmentedMatrix creates an n x 2 matrix from xValues
// The first column is ones (for the intercept) and the second column is xValues
func BuildAugmentedMatrix(xValues []float64) *mat.Dense {
	n := len(xValues)
	XAug := mat.NewDense(n, 2, nil)
	for i := 0; i < n; i++ {
		XAug.Set(i, 0, 1.0)
		XAug.Set(i, 1, xValues[i])
	}
	return XAug
}
EOF

In [40]:
%%bash
cat > golang_linreg/train.go << 'EOF'
package main

import (
	"fmt"
	"time"

	"gonum.org/v1/gonum/mat"
)

// TrainModel trains the linear regression model using the augmented matrix XAug and yValues
// It returns the computed theta (model parameters) and the training duration
func TrainModel(XAug *mat.Dense, yValues []float64) (*mat.VecDense, time.Duration, error) {
	n, _ := XAug.Dims()
	y := mat.NewVecDense(n, yValues)
	trainStart := time.Now()
	var Xt mat.Dense
	Xt.Mul(XAug.T(), XAug)
	var XtInv mat.Dense
	if err := XtInv.Inverse(&Xt); err != nil {
		return nil, 0, fmt.Errorf("matrix inversion failed: %v", err)
	}
	var Xty mat.VecDense
	Xty.MulVec(XAug.T(), y)
	var theta mat.VecDense
	theta.MulVec(&XtInv, &Xty)
	trainDuration := time.Since(trainStart)
	return &theta, trainDuration, nil
}
EOF

In [41]:
%%bash
cat > golang_linreg/predict.go << 'EOF'
package main

import (
	"time"

	"gonum.org/v1/gonum/mat"
)

// Predict uses the trained model (theta) to predict the target value for xNew
// It creates an augmented row for xNew and returns the predicted value and the prediction duration
func Predict(theta *mat.VecDense, xNew float64) (float64, time.Duration) {
	XNewAug := mat.NewDense(1, 2, []float64{1.0, xNew})
	predStart := time.Now()
	var yPred mat.Dense
	yPred.Mul(XNewAug, theta)
	predDuration := time.Since(predStart)
	return yPred.At(0, 0), predDuration
}
EOF

In [42]:
%%bash
cat > golang_linreg/main.go << 'EOF'
package main

import (
	"fmt"
	"math"
	"time" // Required for time.Duration and our dummy usage
)

var _ = time.Nanosecond // Dummy use of time to avoid "imported and not used" error

// normalizeZero returns 0.0 if x is extremely close to zero
func normalizeZero(x float64, tol float64) float64 {
	if math.Abs(x) < tol {
		return 0.0
	}
	return x
}

func main() {
	// Create the dataset
	xValues, yValues, _ := CreateDataset()

	// Build the augmented matrix
	XAug := BuildAugmentedMatrix(xValues)

	// Train the model
	theta, trainDuration, err := TrainModel(XAug, yValues)
	if err != nil {
		fmt.Println(err)
		return
	}

	// Normalize theta values so that very small numbers print as 0.0000
	tol := 1e-12
	thetaData := theta.RawVector().Data
	for i, v := range thetaData {
		thetaData[i] = normalizeZero(v, tol)
	}

	// Print the learned parameters (theta) with 4 decimal places
	fmt.Printf("Learned parameters (theta): [")
	for i, v := range thetaData {
		if i > 0 {
			fmt.Printf(", ")
		}
		fmt.Printf("%.4f", v)
	}
	fmt.Println("]")

	// Print training time in both milliseconds and microseconds
	trainMillis := float64(trainDuration.Milliseconds())
	trainMicros := float64(trainDuration.Microseconds())
	fmt.Printf("Training time: %.6f ms\n", trainMillis)
	fmt.Printf("Training time: %.6f µs\n", trainMicros)

	// Predict for a new data point: x = 6
	prediction, predDuration := Predict(theta, 6.0)
	fmt.Printf("Prediction for x = 6: %.4f\n", prediction)
	predMillis := float64(predDuration.Milliseconds())
	predMicros := float64(predDuration.Microseconds())
	fmt.Printf("Prediction time: %.6f ms\n", predMillis)
	fmt.Printf("Prediction time: %.6f µs\n", predMicros)
}
EOF

In [43]:
%%bash
cd golang_linreg
export GO111MODULE=on
go mod tidy && go mod download
go run .
cd ..

Learned parameters (theta): [0.0000, 2.0000]
Training time: 0.000000 ms
Training time: 208.000000 µs
Prediction for x = 6: 12.0000
Prediction time: 0.000000 ms
Prediction time: 1.000000 µs


go: found gonum.org/v1/gonum/mat in gonum.org/v1/gonum v0.0.0-00010101000000-000000000000
